📦 Cellule 1 : Imports et Configuration


In [31]:
# ========================================
# IMPORTS
# ========================================
import pandas as pd
import pyodbc
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os
from io import BytesIO
from tqdm import tqdm
import warnings
import sys

warnings.filterwarnings('ignore')

print("=" * 70)
print("📊 ETL - CHARGEMENT DE DIM_ACTIVITE_NAF DEPUIS AZURE DATA LAKE")
print("=" * 70)

# ========================================
# CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ========================================
load_dotenv()

# Configuration Azure Data Lake
STORAGE_ACCOUNT_NAME = os.getenv('STORAGE_ACCOUNT_NAME')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')
CONTAINER_NAME_DATA_GOUV = "data-gouv"

# Configuration DWH
DB_SERVER_DWH = 'carter-cash-serveur.database.windows.net'
DB_DATABASE_DWH = 'DWH_E5_projet_AUTO'
DB_USERNAME_DWH = os.getenv('DB_USERNAME')
DB_PASSWORD_DWH = os.getenv('DB_PASSWORD')

print("\n✅ Variables d'environnement chargées")
print(f"   • Storage Account : {STORAGE_ACCOUNT_NAME}")
print(f"   • Container : {CONTAINER_NAME_DATA_GOUV}")
print(f"   • DWH Database : {DB_DATABASE_DWH}")

📊 ETL - CHARGEMENT DE DIM_ACTIVITE_NAF DEPUIS AZURE DATA LAKE

✅ Variables d'environnement chargées
   • Storage Account : datalakecertifimpe
   • Container : data-gouv
   • DWH Database : DWH_E5_projet_AUTO


📦 Cellule 2 : Connexion au Data Lake Azure


In [32]:
print("\n" + "=" * 70)
print("🔗 CONNEXION AU DATA LAKE AZURE")
print("=" * 70)

try:
    # Créer le client Blob Storage
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    
    # Tester la connexion
    container_client = blob_service_client.get_container_client(CONTAINER_NAME_DATA_GOUV)
    
    print(f"✅ Connexion réussie au Data Lake : {STORAGE_ACCOUNT_NAME}")
    print(f"✅ Container : {CONTAINER_NAME_DATA_GOUV}")
    
except Exception as e:
    print(f"❌ Erreur de connexion au Data Lake : {e}")
    raise


🔗 CONNEXION AU DATA LAKE AZURE
✅ Connexion réussie au Data Lake : datalakecertifimpe
✅ Container : data-gouv


📦 Cellule 3 : Connexion au DWH


In [33]:
print("\n" + "=" * 70)
print("🔗 CONNEXION AU DATA WAREHOUSE")
print("=" * 70)

try:
    connection_string_dwh = (
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={DB_SERVER_DWH};'
        f'DATABASE={DB_DATABASE_DWH};'
        f'UID={DB_USERNAME_DWH};'
        f'PWD={DB_PASSWORD_DWH}'
    )
    
    cnxn_dwh = pyodbc.connect(connection_string_dwh)
    cursor_dwh = cnxn_dwh.cursor()
    
    print(f"✅ Connexion réussie au DWH")
    print(f"   • Serveur : {DB_SERVER_DWH}")
    print(f"   • Base de données : {DB_DATABASE_DWH}")
    
except Exception as e:
    print(f"❌ Erreur de connexion au DWH : {e}")
    raise


🔗 CONNEXION AU DATA WAREHOUSE
✅ Connexion réussie au DWH
   • Serveur : carter-cash-serveur.database.windows.net
   • Base de données : DWH_E5_projet_AUTO


📦 Cellule 4 : Téléchargement et Lecture du fichier NAF


In [34]:
print("\n" + "=" * 70)
print("📥 TÉLÉCHARGEMENT DU FICHIER NAF DEPUIS LE DATA LAKE")
print("=" * 70)

# ========================================
# SÉLECTIONNER LE BON FICHIER
# ========================================
# On a trouvé ces fichiers dans la cellule précédente :
#   • data_NAF (dossier - à ignorer)
#   • data_NAF/naf_rev2_20250129.xls
#   • data_NAF/naf_rev2_20250225.xls

# Sélectionner le fichier le plus récent (20250225)
blob_path = "data_NAF/naf_rev2_20250225.xls"

print(f"📂 Fichier sélectionné : {blob_path}")

try:
    # ========================================
    # TÉLÉCHARGER LE FICHIER
    # ========================================
    print(f"\n⏳ Téléchargement du fichier...")
    
    blob_client = blob_service_client.get_blob_client(
        container=CONTAINER_NAME_DATA_GOUV, 
        blob=blob_path
    )
    
    # Vérifier que le blob existe
    if not blob_client.exists():
        raise FileNotFoundError(f"Le fichier {blob_path} n'existe pas")
    
    # Télécharger le contenu
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall()
    
    print(f"✅ Fichier téléchargé : {len(blob_content):,} octets ({len(blob_content)/1024:.2f} KB)")
    
    # ========================================
    # LIRE LE FICHIER EXCEL
    # ========================================
    print("\n⏳ Lecture du fichier Excel...")
    
    # Le fichier .xls nécessite le moteur 'xlrd'
    # Si erreur, installer : pip install xlrd
    df_naf_raw = pd.read_excel(BytesIO(blob_content), engine='xlrd')
    
    print(f"✅ Fichier chargé : {len(df_naf_raw)} lignes × {len(df_naf_raw.columns)} colonnes")
    
    # ========================================
    # AFFICHER LES DONNÉES BRUTES
    # ========================================
    print(f"\n📊 APERÇU DES PREMIÈRES LIGNES (BRUTES) :")
    print("=" * 70)
    print(df_naf_raw.head(20))
    
    print(f"\n📊 COLONNES DÉTECTÉES :")
    print(list(df_naf_raw.columns))
    
    print(f"\n📊 INFO SUR LE DATAFRAME :")
    df_naf_raw.info()
    
except FileNotFoundError as e:
    print(f"❌ Erreur : {e}")
    print("\n💡 SOLUTION :")
    print("   Le fichier n'existe pas à ce chemin.")
    print("   Fichiers disponibles :")
    print("   • data_NAF/naf_rev2_20250129.xls")
    print("   • data_NAF/naf_rev2_20250225.xls")
    raise

except ImportError as e:
    print(f"❌ Erreur : Module 'xlrd' manquant")
    print("\n💡 SOLUTION :")
    print("   Installe le module xlrd pour lire les fichiers .xls :")
    print("   pip install xlrd")
    raise

except Exception as e:
    print(f"❌ Erreur lors du téléchargement/lecture du fichier : {e}")
    print(f"\n🔍 Type d'erreur : {type(e).__name__}")
    print(f"📄 Chemin utilisé : {blob_path}")
    raise


📥 TÉLÉCHARGEMENT DU FICHIER NAF DEPUIS LE DATA LAKE
📂 Fichier sélectionné : data_NAF/naf_rev2_20250225.xls

⏳ Téléchargement du fichier...
✅ Fichier téléchargé : 311,808 octets (304.50 KB)

⏳ Lecture du fichier Excel...
✅ Fichier chargé : 2109 lignes × 5 colonnes

📊 APERÇU DES PREMIÈRES LIGNES (BRUTES) :
    ligne       Code       Intitulés de la  NAF rév. 2, version finale   \
0       1        NaN                                                NaN   
1       2  SECTION A                 AGRICULTURE, SYLVICULTURE ET PÊCHE   
2       3        NaN                                                NaN   
3       4         01  Culture et production animale, chasse et servi...   
4       5        NaN                                                NaN   
5       6       01.1                           Cultures non permanentes   
6       7      01.11  Culture de céréales (à l'exception du riz), de...   
7       8     01.11Z  Culture de céréales (à l'exception du riz), de...   
8       9      01.

In [35]:
# Supprimer toutes les lignes où la colonne "Code" contient des NaN
df_naf_raw = df_naf_raw.dropna(subset=['Code'])



# Réinitialiser l'index
df_naf_raw = df_naf_raw.reset_index(drop=True)

df_naf_filtered = df_naf_raw[df_naf_raw['Code'].str.startswith(('45'))]


df_naf_raw = df_naf_filtered


In [36]:
df_naf_raw

,ligne,Code,"Intitulés de la NAF rév. 2, version finale","Intitulés NAF rév. 2, \nen 65 caractères","Intitulés NAF rév. 2, \nen 40 caractères"
874,1059,45,Commerce et réparation d'automobiles et de mot...,Commerce et réparation d'automobiles et de mot...,Commerce & répar. automobile & motocycle
875,1061,45.1,Commerce de véhicules automobiles,Commerce de véhicules automobiles,Commerce de véhicules automobiles
876,1062,45.11,Commerce de voitures et de véhicules automobil...,Commerce de voitures et de véhicules automobil...,Comm. de voiture & véhicule auto. léger
877,1063,45.11Z,Commerce de voitures et de véhicules automobil...,Commerce de voitures et de véhicules automobil...,Comm. de voiture & véhicule auto. léger
878,1064,45.19,Commerce d'autres véhicules automobiles,Commerce d'autres véhicules automobiles,Commerce d'autres véhicules automobiles
879,1065,45.19Z,Commerce d'autres véhicules automobiles,Commerce d'autres véhicules automobiles,Commerce d'autres véhicules automobiles
880,1067,45.2,Entretien et réparation de véhicules automobiles,Entretien et réparation de véhicules automobiles,Entretien & répar. de véhicule auto.
881,1068,45.20,Entretien et réparation de véhicules automobiles,Entretien et réparation de véhicules automobiles,Entretien & répar. de véhicule auto.
882,1069,45.20A,Entretien et réparation de véhicules automobil...,Entretien et réparation de véhicules automobil...,Entretien & répar. véhicule auto. léger
883,1070,45.20B,Entretien et réparation d'autres véhicules aut...,Entretien et réparation d'autres véhicules aut...,Entretien & répar. autre véhicule auto.


In [37]:
from tqdm import tqdm

print("\n" + "=" * 70)
print("📥 ETL - CHARGEMENT DE DIM_ACTIVITE_NAF")
print("=" * 70)

# ========================================
# ÉTAPE 1 : RENOMMER LES COLONNES
# ========================================
print("\n⏳ Préparation des données...")

df_naf_clean = df_naf_raw.copy()

# Renommer les colonnes pour correspondre au schéma DWH
df_naf_clean = df_naf_clean.rename(columns={
    'Code': 'Code_NAF',
    'Intitulés NAF rév. 2, \nen 65 caractères': 'Intitule_NAF_65',
    'Intitulés NAF rév. 2, \nen 40 caractères': 'Intitule_NAF_40'
})

# ========================================
# ÉTAPE 2 : IDENTIFIER SECTION ET DIVISION
# ========================================
print("⏳ Identification des sections et divisions...")

df_naf_clean['Section'] = ''
df_naf_clean['Division'] = ''

current_section = ''
current_division = ''

for idx, row in df_naf_clean.iterrows():
    code = str(row['Code_NAF']).strip()
    
    # Identifier une SECTION (ex: "SECTION A")
    if code.startswith('SECTION'):
        current_section = code
        current_division = ''
        df_naf_clean.at[idx, 'Section'] = current_section
    
    # Identifier une Division (code à 2 chiffres uniquement, ex: "01", "45")
    elif code.replace('.', '').isdigit() and len(code) == 2:
        current_division = code
        df_naf_clean.at[idx, 'Section'] = current_section
        df_naf_clean.at[idx, 'Division'] = current_division
    
    # Codes avec point (ex: "45.1", "45.11Z")
    else:
        df_naf_clean.at[idx, 'Section'] = current_section
        df_naf_clean.at[idx, 'Division'] = current_division

# ========================================
# ÉTAPE 3 : FILTRER LES LIGNES SECTION
# ========================================
print("⏳ Filtrage des codes NAF valides...")

# Exclure les lignes qui sont uniquement des sections (on garde uniquement les codes NAF)
df_naf_final = df_naf_clean[~df_naf_clean['Code_NAF'].str.startswith('SECTION')].copy()

# Sélectionner uniquement les colonnes nécessaires
df_naf_final = df_naf_final[['Code_NAF', 'Intitule_NAF_65', 'Intitule_NAF_40', 'Section', 'Division']].copy()

# Nettoyer les NaN
df_naf_final = df_naf_final.fillna('')

# Supprimer les doublons sur Code_NAF
df_naf_final = df_naf_final.drop_duplicates(subset=['Code_NAF'])

print(f"✅ {len(df_naf_final)} codes NAF uniques préparés")

# ========================================
# ÉTAPE 4 : CHARGER LES CODES NAF EXISTANTS
# ========================================
print("\n⏳ Chargement des codes NAF déjà présents dans le DWH...")
cursor_dwh.execute("SELECT Code_NAF FROM DIM_ACTIVITE_NAF")
naf_existants = set(row[0] for row in cursor_dwh.fetchall())
print(f"✅ {len(naf_existants)} codes NAF déjà présents dans DIM_ACTIVITE_NAF")

# ========================================
# ÉTAPE 5 : FILTRER LES NOUVEAUX CODES NAF
# ========================================
df_naf_nouveaux = df_naf_final[~df_naf_final['Code_NAF'].isin(naf_existants)]
skip_count = len(df_naf_final) - len(df_naf_nouveaux)

print(f"✅ {len(df_naf_nouveaux)} nouveaux codes NAF à insérer")
print(f"⚠️  {skip_count} codes NAF déjà existants (ignorés)")

# ========================================
# ÉTAPE 6 : INSERTION PAR BATCH
# ========================================
if len(df_naf_nouveaux) > 0:
    print("\n⏳ Insertion des nouveaux codes NAF...")
    
    insert_query = """
    INSERT INTO DIM_ACTIVITE_NAF (
        Code_NAF, Intitule_NAF_65, Intitule_NAF_40, Section, Division
    ) VALUES (?, ?, ?, ?, ?)
    """
    
    batch_size = 300
    total_insert = 0
    tuples = [tuple(x) for x in df_naf_nouveaux.values]
    
    for i in tqdm(range(0, len(tuples), batch_size), desc="Insertion codes NAF", unit="batch"):
        batch = tuples[i:i+batch_size]
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            total_insert += len(batch)
        except Exception as e:
            print(f"\n⚠️  Erreur batch {i//batch_size+1}: {e}")
            # Insérer ligne par ligne en cas d'erreur
            for row in batch:
                try:
                    cursor_dwh.execute(insert_query, row)
                    total_insert += 1
                except Exception as e_detail:
                    print(f"   ❌ Erreur : {e_detail}")
                    print(f"   Code NAF : {row[0]}")
            cnxn_dwh.commit()
    
    print(f"\n✅ Insertion terminée : {total_insert} codes NAF insérés")
else:
    print("\n✅ Aucun nouveau code NAF à insérer")
    total_insert = 0

# ========================================
# ÉTAPE 7 : VÉRIFICATIONS FINALES
# ========================================
print("\n🔍 VÉRIFICATION FINALE")
print("=" * 70)

cursor_dwh.execute("SELECT COUNT(*) FROM DIM_ACTIVITE_NAF")
total_final = cursor_dwh.fetchone()[0]
print(f"📊 Nombre total de codes NAF dans DIM_ACTIVITE_NAF : {total_final}")

# Statistiques par section
print("\n📈 Répartition par section :")
cursor_dwh.execute("""
SELECT 
    Section,
    COUNT(*) as Nb_Codes
FROM DIM_ACTIVITE_NAF
WHERE Section != ''
GROUP BY Section
ORDER BY Section
""")

for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} codes")

# Exemples de codes NAF insérés
print("\n📋 EXEMPLES DE CODES NAF (section automobile) :")
cursor_dwh.execute("""
SELECT TOP 10
    Code_NAF, Intitule_NAF_65, Section, Division
FROM DIM_ACTIVITE_NAF
WHERE Code_NAF LIKE '45%'
ORDER BY Code_NAF
""")

for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]}")
    print(f"     Section: {row[2]} | Division: {row[3]}")

print("\n" + "=" * 70)
print("✅ CHARGEMENT TERMINÉ - DIM_ACTIVITE_NAF")
print("=" * 70)
print(f"   📊 Total de codes NAF : {total_final}")
print(f"   ➕ Nouveaux codes insérés : {total_insert}")
print(f"   ⏭️  Codes déjà existants : {skip_count}")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_ACTIVITE_NAF

⏳ Préparation des données...
⏳ Identification des sections et divisions...
⏳ Filtrage des codes NAF valides...
✅ 18 codes NAF uniques préparés

⏳ Chargement des codes NAF déjà présents dans le DWH...
✅ 0 codes NAF déjà présents dans DIM_ACTIVITE_NAF
✅ 18 nouveaux codes NAF à insérer
⚠️  0 codes NAF déjà existants (ignorés)

⏳ Insertion des nouveaux codes NAF...


Insertion codes NAF: 100%|██████████| 1/1 [00:00<00:00,  1.52batch/s]


✅ Insertion terminée : 18 codes NAF insérés

🔍 VÉRIFICATION FINALE
📊 Nombre total de codes NAF dans DIM_ACTIVITE_NAF : 18

📈 Répartition par section :

📋 EXEMPLES DE CODES NAF (section automobile) :
   • 45 : Commerce et réparation d'automobiles et de motocycles
     Section:  | Division: 45
   • 45.1 : Commerce de véhicules automobiles
     Section:  | Division: 45
   • 45.11 : Commerce de voitures et de véhicules automobiles légers
     Section:  | Division: 45
   • 45.11Z : Commerce de voitures et de véhicules automobiles légers
     Section:  | Division: 45
   • 45.19 : Commerce d'autres véhicules automobiles
     Section:  | Division: 45
   • 45.19Z : Commerce d'autres véhicules automobiles
     Section:  | Division: 45
   • 45.2 : Entretien et réparation de véhicules automobiles
     Section:  | Division: 45
   • 45.20 : Entretien et réparation de véhicules automobiles
     Section:  | Division: 45
   • 45.20A : Entretien et réparation de véhicules automobiles légers
     Sectio